In [ ]:
!pip install geopandas

In [108]:
import time
import json
import fiona
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, LineString, Point
from shapely.geometry import shape
import concurrent.futures

In [2]:
#points = gpd.read_file('points.shp')
#footprint_buildings = gpd.read_file('SF_Buildings_FootPrints.shp')
points = gpd.read_file('../Civil-Work-Bidding-And-Investment-Helper-FootPrint/points.shp')
footprint_buildings = gpd.read_file('../Civil-Work-Bidding-And-Investment-Helper-FootPrint/SF_Buildings_FootPrints.shp')

In [3]:
len(footprint_buildings)

177023

In [4]:
len(points)

7561

In [5]:
points.head()

,full_addre,geometry
0,"3686 17th St, San Francisco, CA 94114",POINT (-122.42798 37.76317)
1,"1254 18th Av, San Francisco, CA 94122",POINT (-122.47579 37.76447)
2,"1059 Union St, San Francisco, CA 94133",POINT (-122.41653 37.79926)
3,"3900 California St, San Francisco, CA 94118",POINT (-122.45952 37.78586)
4,"1084 De Haro St, San Francisco, CA 94107",POINT (-122.40108 37.75608)


In [6]:
#on récupère notre liste de point et on la met dans une serie
points_serie = gpd.GeoSeries(points["geometry"])

In [7]:
#on récupère notre liste de polygon et on la met dans une serie

polygons_serie = gpd.GeoSeries(footprint_buildings["geometry"])

In [8]:
# on met les datas sur le meme CRS (on se base sur celle utilisé pour les polygons des batiments de SF)
points_serie = points_serie.to_crs(polygons_serie.crs)

In [9]:
points_serie.crs

<Geographic 2D CRS: GEOGCS["WGS84(DD)",DATUM["WGS84",SPHEROID["WGS84", ...>
Name: WGS84(DD)
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: WGS84
- Ellipsoid: WGS84
- Prime Meridian: Greenwich

In [10]:
splited_points = np.split(points_serie, [500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000], axis=0)

In [11]:
len(splited_points)

15

In [12]:
def get_polygon_list(points):
    polygon_list = []
    for i in range(len(points)):
        mask = polygons_serie.contains(points_serie[i])
        polygon_list.append(polygons_serie[mask])
        print(f'{len(polygon_list)}\n')
    return polygon_list

In [13]:
polygons = []
start_time = time.time()
with concurrent.futures.ThreadPoolExecutor() as e:
    polygons = list(e.map(get_polygon_list, splited_points)) 
print("--- %s seconds ---" % (time.time() - start_time))

1

1

1

1

1

1

1

1

1

1

1

1

1

1

1

2

2

2

2

2

2

2

2

2

2

2

2

2

2

2

3

3

3

3

3

3

3

3

3

3

3

3

3

3

3

4

4

4

4

4

4

4

4

4

4

4

4

4

4

4

5

5

5

5

5
5


5

5

5

5

5

5

5

5

5

6

6

6

6

6

6

6

6

6

6

6

6

6

6

6

7

7

7

7

7

7

7
7


7

7

7

7

7

7

7

8

8

8

8

8

8

8

8

8

8

8

8

8

8

8

9

9

9

9

9
9


9

9

9

9

9

9
9


9

9

10

10

10

10

10

10

10

10

10

10

10

10

10

10

10

11

11

11

11

11

11

11

11

11

11

11

11

11

11

11

12

12

12

12
12


12

12

12

12

12

12

12

12

12

12

13

13

13

13

13

13

13

13

13

13

13

13

13

13

13

14

14

14

14

14

14

14

14

14

14

14

14

14

14

15
14


15

15

15

15

15

15

15

15

15

15

15

15

15

16

16

16

15

16

16

16

16

16

16

16

16

16

16

16

17

17

17

16

17

17

17

17

17

17

17

17

17

17

17

18

18

18

17

18

18

18

18

18

18

18

18

18

18

18

19

19

19

19

18

19

19

19

19

19

19

19

19

19

19


In [14]:
len(polygons)

15

In [15]:
geometry_polygons_list = [feature for result in polygons for feature in result]

In [16]:
geometry_polygons_list[0].crs

<Geographic 2D CRS: GEOGCS["WGS84(DD)",DATUM["WGS84",SPHEROID["WGS84", ...>
Name: WGS84(DD)
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: WGS84
- Ellipsoid: WGS84
- Prime Meridian: Greenwich

Convert polygon to crs 2227 to work with foots metrics in the area of San Francisco
[source](https://maps.omniscale.com/en/openstreetmap/epsg-2227)

[Example map in EPSG:2227 – NAD83 / California zone 3 (ftUS)](https://maps.omniscale.com/en/openstreetmap/epsg-2227)

Area may be invalid for a geographic CRS using degrees as units; 
so we use GeoSeries.to_crs() to project geometries to a planar CRS before calculating the area

In [17]:
# transform the polygons with foots metrics based on EPSG:2227
print('transform the polygons with foots metrics based on EPSG:2227')
geometry_polygons_list = [geometry.to_crs(epsg=2227) for geometry in geometry_polygons_list]

transform the polygons with foots metrics based on EPSG:2227


In [19]:
# calculate the area of each polygons
#I n metric terms a square foot is a square with sides 0.3048 metres in length. One square foot is the equivalent to 0.09290304 square metres.
print('calculate the area of each polygons')
geometry_area_m2_list = [geometry.area * 0.09290304 for geometry in geometry_polygons_list]

transform the polygons with foots metrics based on EPSG:2227


In [85]:
geometry_polygons_list[:5]

[54695    POLYGON ((6004384.442 2106033.407, 6004380.959...
 Name: geometry, dtype: geometry,
 16204    POLYGON ((5990594.491 2106871.242, 5990597.672...
 Name: geometry, dtype: geometry,
 GeoSeries([], Name: geometry, dtype: geometry),
 6844    POLYGON ((5995398.966 2114500.145, 5995398.030...
 Name: geometry, dtype: geometry,
 37109    POLYGON ((6012157.403 2103366.943, 6012157.499...
 Name: geometry, dtype: geometry]

In [86]:
geometry_area_m2_list[:5]

[54695    147.915331
 dtype: float64,
 16204    253.922359
 dtype: float64,
 Series([], dtype: float64),
 6844    504.669823
 dtype: float64,
 37109    167.842843
 dtype: float64]

On évite de stocker l'objet geometrie et on récupère uniquement la valeur numérique

In [87]:
area_m2_list = []
for index in range(len(geometry_area_m2_list)):
    try:
        area_m2_list.append(geometry_area_m2_list[index].values[0])
    except IndexError:
        area_m2_list.append(np.nan)

area_m2_list[:5]

[147.9153310250603,
 253.92235897283538,
 nan,
 504.6698229406359,
 167.8428429089405]

On évite de stocker l'objet geometrie et on récupère uniquement le shapely

In [82]:
polygons_list = []
for index in range(len(geometry_polygons_list)):
    try:
        polygons_list.append(geometry_polygons_list[index].values[0])
    except IndexError:
        polygons_list.append(np.nan)

polygons_list[:5]

 nan,

In [89]:
points['polygon'] = polygons_list
points['area_m2'] = area_m2_list

/home/shinaki/workspace/GitHub/Civil-Work-Bidding-And-Investment-Helper/venv/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:1983: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values


In [98]:
points.head()

,full_addre,geometry,polygon,area_m2
0,"3686 17th St, San Francisco, CA 94114",POINT (-122.42798 37.76317),"POLYGON ((6004384.44156398 2106033.407080037, ...",147.915331
1,"1254 18th Av, San Francisco, CA 94122",POINT (-122.47579 37.76447),"POLYGON ((5990594.490913103 2106871.242348627,...",253.922359
2,"1059 Union St, San Francisco, CA 94133",POINT (-122.41653 37.79926),NaN,NaN
3,"3900 California St, San Francisco, CA 94118",POINT (-122.45952 37.78586),POLYGON ((5995398.966155565 2114500.1454920806...,504.669823
4,"1084 De Haro St, San Francisco, CA 94107",POINT (-122.40108 37.75608),POLYGON ((6012157.402672291 2103366.9430118585...,167.842843


In [99]:
d = {'address': points["full_addre"], 'point': points["geometry"], 'polygon': points["polygon"], 'area_m2': points['area_m2']}
df = pd.DataFrame(data = d)

In [100]:
df.head()

,address,point,polygon,area_m2
0,"3686 17th St, San Francisco, CA 94114",POINT (-122.42798 37.76317),"POLYGON ((6004384.44156398 2106033.407080037, ...",147.915331
1,"1254 18th Av, San Francisco, CA 94122",POINT (-122.47579 37.76447),"POLYGON ((5990594.490913103 2106871.242348627,...",253.922359
2,"1059 Union St, San Francisco, CA 94133",POINT (-122.41653 37.79926),NaN,NaN
3,"3900 California St, San Francisco, CA 94118",POINT (-122.45952 37.78586),POLYGON ((5995398.966155565 2114500.1454920806...,504.669823
4,"1084 De Haro St, San Francisco, CA 94107",POINT (-122.40108 37.75608),POLYGON ((6012157.402672291 2103366.9430118585...,167.842843


In [113]:
dataset_v6 = pd.read_csv('../documents/Building_Permits_v6.csv')

In [114]:
dataset_v6.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Street Suffix,Description,...,Duration_construction_days,Process_time_days,Year,Est_Cost_Infl,Rev_Cost_Infl,Est_Cost_Infl_log10,Est_Cost_Infl_loge,lat_lon,full_address,geometry
0,9111378,2,new construction wood frame,1991-06-28,3566,037,3686,17th,St,erect a four story three unit residential bldg,...,294.0,521.0,1991,4.973752e+05,696325.256975,5.696684,13.117100,-4623.268159,"3686 17th St, San Francisco, CA 94114",NaN
1,8707430,2,new construction wood frame,1987-05-29,1733,046,1254,18th,Av,NaN,...,261.0,236.0,1987,9.398079e+05,NaN,5.973039,13.753431,-4625.233438,"1254 18th Av, San Francisco, CA 94122","{""type"": ""Polygon"", ""coordinates"": [[[-122.476..."
2,7711072S,2,new construction wood frame,1977-10-19,0121,032,1059,Union,St,to erect 12 unit apartment,...,827.0,2477.0,1977,7.476268e+06,NaN,6.873685,15.827244,-4627.254130,"1059 Union St, San Francisco, CA 94133",NaN
3,7912377,2,new construction wood frame,1979-12-01,1361,064,3900,California,St,NaN,...,401.0,1150.0,1979,2.948572e+06,NaN,6.469612,14.896831,-4627.238361,"3900 California St, San Francisco, CA 94118","{""type"": ""Polygon"", ""coordinates"": [[[-122.459..."
4,8010466,2,new construction wood frame,1980-11-25,4159,074,1084,De Haro,St,NaN,...,333.0,1350.0,1980,6.922231e+05,NaN,5.840246,13.447664,-4621.384747,"1084 De Haro St, San Francisco, CA 94107",NaN


In [117]:
dataset_v7 = dataset_v6

In [120]:
df.head()

,address,point,polygon,area_m2
0,"3686 17th St, San Francisco, CA 94114",POINT (-122.42798 37.76317),"POLYGON ((6004384.44156398 2106033.407080037, ...",147.915331
1,"1254 18th Av, San Francisco, CA 94122",POINT (-122.47579 37.76447),"POLYGON ((5990594.490913103 2106871.242348627,...",253.922359
2,"1059 Union St, San Francisco, CA 94133",POINT (-122.41653 37.79926),NaN,NaN
3,"3900 California St, San Francisco, CA 94118",POINT (-122.45952 37.78586),POLYGON ((5995398.966155565 2114500.1454920806...,504.669823
4,"1084 De Haro St, San Francisco, CA 94107",POINT (-122.40108 37.75608),POLYGON ((6012157.402672291 2103366.9430118585...,167.842843


In [128]:
dataset_v7['point'] = df['point']
dataset_v7['polygon'] = df['polygon']
dataset_v7['area_m2'] = df['area_m2']
dataset_v7.rename(columns={"full_address": "address"}, inplace=True)

In [129]:
dataset_v7.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Street Suffix,Description,...,Est_Cost_Infl,Rev_Cost_Infl,Est_Cost_Infl_log10,Est_Cost_Infl_loge,lat_lon,address,geometry,point,polygon,area_m2
0,9111378,2,new construction wood frame,1991-06-28,3566,037,3686,17th,St,erect a four story three unit residential bldg,...,4.973752e+05,696325.256975,5.696684,13.117100,-4623.268159,"3686 17th St, San Francisco, CA 94114",NaN,POINT (-122.42798 37.76317),"POLYGON ((6004384.44156398 2106033.407080037, ...",147.915331
1,8707430,2,new construction wood frame,1987-05-29,1733,046,1254,18th,Av,NaN,...,9.398079e+05,NaN,5.973039,13.753431,-4625.233438,"1254 18th Av, San Francisco, CA 94122","{""type"": ""Polygon"", ""coordinates"": [[[-122.476...",POINT (-122.47579 37.76447),"POLYGON ((5990594.490913103 2106871.242348627,...",253.922359
2,7711072S,2,new construction wood frame,1977-10-19,0121,032,1059,Union,St,to erect 12 unit apartment,...,7.476268e+06,NaN,6.873685,15.827244,-4627.254130,"1059 Union St, San Francisco, CA 94133",NaN,POINT (-122.41653 37.79926),NaN,NaN
3,7912377,2,new construction wood frame,1979-12-01,1361,064,3900,California,St,NaN,...,2.948572e+06,NaN,6.469612,14.896831,-4627.238361,"3900 California St, San Francisco, CA 94118","{""type"": ""Polygon"", ""coordinates"": [[[-122.459...",POINT (-122.45952 37.78586),POLYGON ((5995398.966155565 2114500.1454920806...,504.669823
4,8010466,2,new construction wood frame,1980-11-25,4159,074,1084,De Haro,St,NaN,...,6.922231e+05,NaN,5.840246,13.447664,-4621.384747,"1084 De Haro St, San Francisco, CA 94107",NaN,POINT (-122.40108 37.75608),POLYGON ((6012157.402672291 2103366.9430118585...,167.842843


In [130]:
dataset_v7.to_csv('../documents/Building_Permits_v7.csv')